In [ ]:
ZILLIZDB_USERNAME = "db_f551eced4483bcc"
ZILLIZDB_PASSWORD = "Yn9%Ts4tgkDm/ta/"
ZILLIZDB_HOST = "in05-2f51803fc57ed00.serverless.gcp-us-west1.cloud.zilliz.com"
ZILLIZDB_PORT = "19531"
ZILLIZDB_TOKEN = "f41eb4a7c305fd31720650e0ea5c037dae612fc47865cad5c5edd5429c14e1e96582b2ac87c0abcd26241fa4b3611c3054573682"
ZILLIZDB_COLLECTION_NAME_DOCUMENTS = "argo_documents_hs"

In [ ]:
import pandas as pd
from pymilvus import (
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    AnnSearchRequest,
    RRFRanker,
    connections,
)
from pymilvus import MilvusClient
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

In [ ]:
ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
dense_dim = ef.dim["dense"]

In [ ]:
dense_dim

In [ ]:
df = pd.read_excel("../data/knowledge.xlsx")
df.shape

In [ ]:
def get_content_embedding_sparse(content):
    embedding_sparse_vector = ef([content])
    embedding_sparse_vector = embedding_sparse_vector["sparse"]
    return embedding_sparse_vector

In [ ]:
def get_content_embedding_milvus(content):
    embedding_sparse_vector = ef([content])
    return embedding_sparse_vector

In [ ]:
embed_content = get_content_embedding_milvus("hihi")

In [ ]:
res = embed_content["dense"][0]

In [ ]:
type(res)

In [ ]:
df = df[:4]

In [ ]:
docs = ["hihi", "haha"]

In [ ]:
docs_embeddings = ef(docs)

In [ ]:
docs_embeddings

In [ ]:
docs_embeddings['sparse'][0]

In [ ]:
docs_embeddings['dense']

In [ ]:
documents = []
for index, row in df.iterrows():
    embed_content = get_content_embedding_milvus(row["content"])
    documents.append({
        "id": row["id"],
        "title": row["title"],
        "content": row["content"],
        "dense_vector": embed_content["dense"][0],
        "sparse_vector": embed_content["sparse"][0],
    })
len(documents)

In [ ]:
connections.connect(
    uri=f'https://{ZILLIZDB_HOST}',
    token=ZILLIZDB_TOKEN
)

In [ ]:
fields = [
    FieldSchema(name="id", dtype=DataType.VARCHAR, is_primary=True, max_length=100),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=65535),
    FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=65535),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
]
schema = CollectionSchema(fields, "")
col = Collection("argo_documents_hs", schema)

In [ ]:
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
dense_index = {"index_type": "FLAT", "metric_type": "COSINE"}
col.create_index("sparse_vector", sparse_index)
col.create_index("dense_vector", dense_index)

In [32]:
col.insert(documents)

(insert count: 4, delete count: 0, upsert count: 0, timestamp: 451298089810001922, success count: 4, err count: 0, cost: 7)

In [33]:
collection = Collection(name=ZILLIZDB_COLLECTION_NAME_DOCUMENTS)

In [35]:
query = "tín chỉ carbon là gì?"
embed_content = get_content_embedding_milvus(query)
sparse_req = AnnSearchRequest(
    embed_content["sparse"][0], "sparse_vector", {"metric_type": "IP"}, limit=2
)
dense_req = AnnSearchRequest(
    embed_content["dense"][0], "dense_vector", {"metric_type": "COSINE"}, limit=2
)

In [36]:
search_results = collection.hybrid_search(
    reqs=[sparse_req, dense_req],
    rerank=RRFRanker(),
    limit=2,
    # output_fields=["id", "title", "content"]
)

Unexpected error: [hybrid_search], object of type 'numpy.float32' has no len(), <Time: {'RPC start': '2024-07-21 19:41:10.790975', 'Exception': '2024-07-21 19:41:10.790975'}>


MilvusException: <MilvusException: (code=1, message=Unexpected error, message=<object of type 'numpy.float32' has no len()>)>